In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_palette('husl')
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [11]:
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [12]:
df = pd.read_csv(filepath_or_buffer="./mpc_data_1_formatted.csv")
df.describe()

,Time,i_k1_d,i_k1_q,Io_d,IO_q,theta_d,theta_q,vdc,sa,sb,sc,STATES2
count,5002.000000,5002.000000,5.002000e+03,5002.000000,5002.000000,5.002000e+03,5.002000e+03,5002.000000,5002.000000,5002.000000,5002.000000,5002.000000
mean,0.049990,10.903468,-1.189124e-15,10.842085,-0.009788,1.806367e-05,-3.990223e-04,349.993150,3.690924,3.689324,3.624550,409.610156
std,0.028882,3.576909,1.269293e-14,3.558902,0.134322,7.070372e-01,7.073176e-01,0.005649,1.877053,1.877641,1.900173,206.470250
min,0.000000,-14.142136,-9.800000e-14,0.000000,-1.570372,-1.000000e+00,-1.000000e+00,349.985635,1.000000,1.000000,1.000000,111.000000
25%,0.024985,7.071068,-5.000000e-15,7.074277,-0.098465,-7.062176e-01,-7.079948e-01,349.987131,1.000000,1.000000,1.000000,115.000000
50%,0.049990,14.142136,-1.000000e-15,13.985045,-0.005770,1.000000e-15,-3.500000e-15,349.993029,5.000000,5.000000,5.000000,551.000000
75%,0.074995,14.142136,3.000000e-15,14.153253,0.087342,7.062176e-01,7.062176e-01,349.999990,5.000000,5.000000,5.000000,555.000000
max,0.100000,35.355339,1.170000e-13,14.387849,0.494396,1.000000e+00,1.000000e+00,350.006757,5.000000,5.000000,5.000000,555.000000


In [13]:
df.head()

,Time,i_k1_d,i_k1_q,Io_d,IO_q,theta_d,theta_q,vdc,sa,sb,sc,STATES2
0,0.00000,28.284271,-2.000000e-15,0.000000,0.000000,-0.000000,-1.000000,350.000000,5,5,5,555
1,0.00002,-14.142136,5.000000e-15,0.338512,-0.198858,0.007540,-0.999972,349.999620,5,5,1,551
2,0.00004,14.142136,-1.000000e-15,0.050710,-0.030306,0.015079,-0.999886,350.000072,1,1,5,115
3,0.00006,7.071068,5.000000e-15,0.335126,0.194171,0.022618,-0.999744,349.999625,1,1,1,111
4,0.00008,7.071068,-2.100000e-14,0.722757,0.013021,0.030155,-0.999545,349.999404,5,5,1,551


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5002 entries, 0 to 5001
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Time     5002 non-null   float64
 1   i_k1_d   5002 non-null   float64
 2   i_k1_q   5002 non-null   float64
 3   Io_d     5002 non-null   float64
 4   IO_q     5002 non-null   float64
 5   theta_d  5002 non-null   float64
 6   theta_q  5002 non-null   float64
 7   vdc      5002 non-null   float64
 8   sa       5002 non-null   int64  
 9   sb       5002 non-null   int64  
 10  sc       5002 non-null   int64  
 11  STATES2  5002 non-null   int64  
dtypes: float64(8), int64(4)
memory usage: 469.1 KB


In [15]:
print(df["STATES2"].value_counts())

555    2205
551    1158
115    1077
111     560
511       2
Name: STATES2, dtype: int64


In [16]:
labelencoder = LabelEncoder()
df["STATES2_LABEL"] = labelencoder.fit_transform(df.values[:, 11])
df.head()

,Time,i_k1_d,i_k1_q,Io_d,IO_q,theta_d,theta_q,vdc,sa,sb,sc,STATES2,STATES2_LABEL
0,0.00000,28.284271,-2.000000e-15,0.000000,0.000000,-0.000000,-1.000000,350.000000,5,5,5,555,4
1,0.00002,-14.142136,5.000000e-15,0.338512,-0.198858,0.007540,-0.999972,349.999620,5,5,1,551,3
2,0.00004,14.142136,-1.000000e-15,0.050710,-0.030306,0.015079,-0.999886,350.000072,1,1,5,115,1
3,0.00006,7.071068,5.000000e-15,0.335126,0.194171,0.022618,-0.999744,349.999625,1,1,1,111,0
4,0.00008,7.071068,-2.100000e-14,0.722757,0.013021,0.030155,-0.999545,349.999404,5,5,1,551,3


In [17]:
y = np_utils.to_categorical(df["STATES2_LABEL"])
X = df[["Time","i_k1_d","i_k1_q","Io_d","IO_q","theta_d","theta_q","vdc","sa","sb","sc"]]
print(X.shape)

(5002, 11)


In [18]:
scaler = MinMaxScaler()
scaler.fit(X)

X = scaler.transform(X=X)
print(X)

[[0.00000000e+00 8.57142857e-01 4.46511628e-01 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]
 [2.00000000e-04 0.00000000e+00 4.79069767e-01 ... 1.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [4.00000000e-04 5.71428571e-01 4.51162791e-01 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 ...
 [9.99600000e-01 5.71428571e-01 4.60465116e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [9.99800000e-01 5.71428571e-01 4.65116279e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 5.71428571e-01 4.65116279e-01 ... 1.00000000e+00
  1.00000000e+00 1.00000000e+00]]


In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(3501, 11)
(3501, 5)
(1501, 11)
(1501, 5)


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout 

model = Sequential()
model.add(Dense(1024,input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy',get_f1])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              12288     
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 1285      
Total params: 669,701
Trainable params: 669,701
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),batch_size=100,epochs=10,verbose=1)

Epoch 1/10
36/36 [==============================] - 1s 21ms/step - loss: 0.2599 - accuracy: 0.9235 - get_f1: 0.8664 - val_loss: 0.0093 - val_accuracy: 0.9993 - val_get_f1: 0.9994
Epoch 2/10
36/36 [==============================] - 0s 13ms/step - loss: 0.0041 - accuracy: 0.9997 - get_f1: 0.9997 - val_loss: 0.0068 - val_accuracy: 0.9993 - val_get_f1: 0.9994
Epoch 3/10
36/36 [==============================] - 0s 14ms/step - loss: 0.0030 - accuracy: 0.9997 - get_f1: 0.9997 - val_loss: 0.0043 - val_accuracy: 0.9993 - val_get_f1: 0.9994
Epoch 4/10
36/36 [==============================] - 0s 14ms/step - loss: 0.0020 - accuracy: 0.9997 - get_f1: 0.9997 - val_loss: 0.0023 - val_accuracy: 0.9993 - val_get_f1: 0.9994
Epoch 5/10
36/36 [==============================] - 0s 13ms/step - loss: 0.0012 - accuracy: 0.9997 - get_f1: 0.9997 - val_loss: 7.1606e-04 - val_accuracy: 0.9993 - val_get_f1: 0.9997
Epoch 6/10
36/36 [==============================] - 0s 13ms/step - loss: 8.4611e-04 - accuracy: 0.999

In [22]:
prediction = model.predict(X_test)
y_label = np.argmax(y_test,axis=1)
predict_label = np.argmax(prediction,axis=1)

print(predict_label)
print(labelencoder.inverse_transform(predict_label))

from sklearn.metrics import accuracy_score as sk_accuracy_score
from sklearn.metrics import f1_score as sk_f1_score
from sklearn.metrics import precision_score as sk_precision_score
from sklearn.metrics import recall_score as sk_recall_score
from sklearn.metrics import confusion_matrix as sk_confusion_matrix

print("sk_accuracy_score :",sk_accuracy_score(y_label, predict_label))
print("sk_f1_score :", sk_f1_score(y_label, predict_label,average="macro"))
print("sk_precision_score :", sk_precision_score(y_label, predict_label,average="macro"))
print("sk_recall_score :" ,sk_recall_score(y_label, predict_label,average="macro"))    

[4 0 4 ... 4 3 0]
[555. 111. 555. ... 555. 551. 111.]
sk_accuracy_score : 1.0
sk_f1_score : 1.0
sk_precision_score : 1.0
sk_recall_score : 1.0


In [23]:
custom_x = np.array([[0.100000000000000,14.142135619999999,0.000000000000002,13.999457350000000,0.136079291000000,-0.000000000000009,-1.000000000000000,349.999989500000027,5,5,5]])
custom_x_normalized = scaler.transform(custom_x)
custom_predict = model.predict(custom_x_normalized)
custom_predict_label = np.argmax(custom_predict,axis=1)

print(custom_predict_label)
print(labelencoder.inverse_transform(custom_predict_label))

[4]
[555.]
